# MNIST handwritten digits classification with nearest neighbors 

In this notebook, we'll use [nearest-neighbor classifiers](https://docs.rapids.ai/api/cuml/stable/api.html#id18) to classify MNIST digits using a GPU and the [RAPIDS](https://rapids.ai/) libraries (cudf, cuml).

**Note that a GPU is required with this notebook.**

This version of the notebook has been tested with RAPIDS version 0.15.

First, the needed imports. 

In [ ]:
%matplotlib inline

from pml_utils import get_mnist, show_failures

import cudf
import numpy as np
import pandas as pd

from cuml.neighbors import KNeighborsClassifier
from cuml import __version__ as cuml_version

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn import __version__ as sklearn_version

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

print('Using cudf version:', cudf.__version__)
print('Using cuml version:', cuml_version)
print('Using sklearn version:', sklearn_version)

Then we load the MNIST data. First time we need to download the data, which can take a while. The data is stored as Numpy arrays in host (CPU) memory.

We also convert `y_train` and `y_test` from strings to integers.

In [ ]:
X_train, y_train, X_test, y_test = get_mnist('MNIST')
y_train = y_train.astype(int)
y_test = y_test.astype(int)

print()
print('MNIST data loaded: train:',len(X_train),'test:',len(X_test))
print('X_train:', type(X_train), 'shape:', X_train.shape)
print('y_train:', type(y_train), 'shape:', y_train.shape, y_train.dtype)
print('X_test:', type(X_test), 'shape:', X_test.shape)
print('y_test:', type(y_test), 'shape:', y_test.shape)

The training data (`X_train`) is a matrix of size (60000, 784), i.e. it consists of 60000 digits expressed as 784 sized vectors (28x28 images flattened to 1D). `y_train` is a 60000-dimensional vector containing the correct class (i.e. one of: 0, 1, ..., 9) for each training digit.

Let's take a closer look. Here are the first 10 training digits:

In [ ]:
pltsize=1
plt.figure(figsize=(10*pltsize, pltsize))

for i in range(10):
    plt.subplot(1,10,i+1)
    plt.axis('off')
    plt.imshow(X_train[i,:].reshape(28, 28), cmap="gray")
    plt.title('Class: '+y_train[i])

Let's then convert our training and test data to cuDF DataFrames in device (GPU) memory. 

We will later evaluate the test data as Numpy arrays, so we do not need to convert `y_test`. 

In [ ]:
cu_X_train = cudf.DataFrame.from_pandas(pd.DataFrame(X_train))
cu_y_train = cudf.DataFrame.from_pandas(pd.DataFrame(y_train))
cu_X_test  = cudf.DataFrame.from_pandas(pd.DataFrame(X_test))

print('cu_X_train:', type(cu_X_train), 'shape:', cu_X_train.shape)
print('cu_y_train:', type(cu_y_train), 'shape:', cu_y_train.shape)
print('cu_X_test:', type(cu_X_test), 'shape:', cu_X_test.shape)

## 1-NN classifier

### Initialization

Let's create first a 1-NN classifier.  Note that with nearest-neighbor classifiers there is no internal (parameterized) model and therefore no learning required.  Instead, calling the `fit()` function simply stores the samples of the training data in a suitable data structure.

In [ ]:
%%time

n_neighbors = 1
cu_clf = KNeighborsClassifier(n_neighbors=n_neighbors)
cu_clf.fit(cu_X_train, cu_y_train)

### Inference

We can now classify the test samples with our classifier. With a GPU, classifying all 10000 samples should be rather fast. 

We also convert the predictions to a Numpy array in host memory with `values_host`.


In [ ]:
%%time

predictions = cu_clf.predict(cu_X_test).values_host.flatten()

The accuracy of the classifier:

In [ ]:
print('Predicted', len(predictions), 'digits with accuracy:', 
      accuracy_score(y_test, predictions))

#### Confusion matrix

We can compute the confusion matrix to see which digits get mixed the most:

In [ ]:
labels=range(10)
print('Confusion matrix (rows: true classes; columns: predicted classes):'); print()
cm=confusion_matrix(y_test, predictions, labels=labels)
print(cm); print()

Plotted as an image:

In [ ]:
plt.matshow(cm, cmap=plt.cm.gray)
plt.xticks(range(10))
plt.yticks(range(10))
plt.grid(None)
plt.show()

#### Accuracy, precision and recall

Classification accuracy for each class:

In [ ]:
for i,j in enumerate(cm.diagonal()/cm.sum(axis=1)): print("%d: %.4f" % (i,j))

Precision and recall for each class:

In [ ]:
print(classification_report(y_test, predictions, labels=labels))

#### Failure analysis

We can also inspect the results in more detail. Let's use the `show_failures()` helper function (defined in `pml_utils.py`) to show the wrongly classified test digits.

The helper function is defined as:

```
show_failures(predictions, y_test, X_test, trueclass=None, predictedclass=None, maxtoshow=10)
```

where:
- `predictions` is a vector with the predicted classes for each test set image
- `y_test` the _correct_ classes for the test set images
- `X_test` the test set images
- `trueclass` can be set to show only images for a given correct (true) class
- `predictedclass` can be set to show only images which were predicted as a given class
- `maxtoshow` specifies how many items to show


In [ ]:
show_failures(predictions, y_test, X_test)

We can use `show_failures()` to inspect failures in more detail. For example:

* show failures in which the true class was "5":

In [ ]:
show_failures(predictions, y_test, X_test, trueclass=5)

* show failures in which the prediction was "0":

In [ ]:
show_failures(predictions, y_test, X_test, predictedclass=0)

* show failures in which the true class was "0" and the prediction was "2":

In [ ]:
show_failures(predictions, y_test, X_test, trueclass=0, predictedclass=2)

## Model tuning

Try to modify the nearest-neighbor classifier. Things to try include using more than one neighbor and adding weights to the neighbors (if supported).  See the documentation for cuml [KNeighborsClassifier](https://docs.rapids.ai/api/cuml/stable/api.html#id18).